# Tier 2. Module 6 - Data Engineering. Homework


## Lessons 01-03. Data analysis in PySpark

You will need to perform various data processing operations in Spark, as well as develop your analytical and programming skills. This task will help you learn the basic concepts and syntax of working with Apache Spark, and learn how to apply this knowledge in practice for data processing.

### Input data

1. users.csv — user data:
* user_id (unique user identifier)
* name (user name)
* age (user age)
* email (email address)

2. purchases.csv — purchase data:
* purchase_id (unique purchase identifier)
* user_id (user identifier who made the purchase)
* product_id (unique product identifier)
* date (purchase date)
* quantity (number of units of the purchased product)

3. products.csv — product information:
* product_id (unique product identifier)
* product_name (product name)
* category (product category)
* price (unit price of the product)

### Technical task

#### 1. Download and read each CSV file as a separate DataFrame

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, round

spark = SparkSession.builder.appName("DataProcessing").getOrCreate()
type(spark)

pyspark.sql.session.SparkSession

In [4]:
users_df = spark.read.csv("input_data/users.csv", header=True, inferSchema=True)
purchases_df = spark.read.csv("input_data/purchases.csv", header=True, inferSchema=True)
products_df = spark.read.csv("input_data/products.csv", header=True, inferSchema=True)

In [8]:
users_df.show(5)

+-------+------+---+-----------------+
|user_id|  name|age|            email|
+-------+------+---+-----------------+
|      1|User_1| 45|user1@example.com|
|      2|User_2| 48|user2@example.com|
|      3|User_3| 36|user3@example.com|
|      4|User_4| 46|user4@example.com|
|      5|User_5| 29|user5@example.com|
+-------+------+---+-----------------+
only showing top 5 rows



In [9]:
purchases_df.show(5)

+-----------+-------+----------+----------+--------+
|purchase_id|user_id|product_id|      date|quantity|
+-----------+-------+----------+----------+--------+
|          1|     52|         9|2022-01-01|       1|
|          2|     93|        37|2022-01-02|       8|
|          3|     15|        33|2022-01-03|       1|
|          4|     72|        42|2022-01-04|       9|
|          5|     61|        44|2022-01-05|       6|
+-----------+-------+----------+----------+--------+
only showing top 5 rows



In [10]:
products_df.show(5)

+----------+------------+-----------+-----+
|product_id|product_name|   category|price|
+----------+------------+-----------+-----+
|         1|   Product_1|     Beauty|  8.3|
|         2|   Product_2|       Home|  8.3|
|         3|   Product_3|Electronics|  9.2|
|         4|   Product_4|Electronics|  2.6|
|         5|   Product_5|Electronics|  9.4|
+----------+------------+-----------+-----+
only showing top 5 rows



#### 2. Clean up the data by removing any rows with missing values

In [11]:
users_df = users_df.dropna()
purchases_df = purchases_df.dropna()
products_df = products_df.dropna()

#### 3. Determine the total amount of purchases for each product category

In [29]:
purchases_products_df = purchases_df.join(products_df, "product_id", "inner")

purchases_by_category = purchases_products_df.groupBy("category").agg(
    round(sum(col("quantity") * col("price")), 2).alias("total_purchase_amount")
)
purchases_by_category.show()

+-----------+---------------------+
|   category|total_purchase_amount|
+-----------+---------------------+
|       Home|               1523.5|
|     Sports|               1802.5|
|Electronics|               1174.8|
|   Clothing|                790.3|
|     Beauty|                459.9|
+-----------+---------------------+



#### 4. Determine the amount of purchases for each product category for the age group 18 to 25 inclusive

In [31]:
purchases_products_users_df = purchases_products_df.join(users_df, "user_id", "inner")

purchases_18_25 = purchases_products_users_df.filter(
    (col("age") > 18) & (col("age") <= 25)
)
purchases_by_category_18_25 = purchases_18_25.groupBy("category").agg(
    round(sum(col("quantity") * col("price")), 2).alias("total_purchases_18_25")
)
purchases_by_category_18_25.show()

+-----------+---------------------+
|   category|total_purchases_18_25|
+-----------+---------------------+
|       Home|                174.7|
|     Sports|                310.5|
|Electronics|                176.0|
|   Clothing|                245.0|
|     Beauty|                 41.4|
+-----------+---------------------+



#### 5. Determine the share of purchases for each product category of the total spending for the age group 18 to 25

In [ ]:
purchases_share_df = purchases_by_category_18_25.join(purchases_by_category, "category")

purchases_share_df = purchases_share_df.withColumn(
    "spending_share_18_25",
    round((col("total_purchases_18_25") / col("total_purchase_amount")) * 100, 2),
)
purchases_share_df.show()

+-----------+---------------------+---------------------+--------------------+
|   category|total_purchases_18_25|total_purchase_amount|spending_share_18_25|
+-----------+---------------------+---------------------+--------------------+
|       Home|                174.7|               1523.5|               11.47|
|     Sports|                310.5|               1802.5|               17.23|
|Electronics|                176.0|               1174.8|               14.98|
|   Clothing|                245.0|                790.3|                31.0|
|     Beauty|                 41.4|                459.9|                 9.0|
+-----------+---------------------+---------------------+--------------------+



#### 6. Select the 3 product categories with the highest percentage of spending by consumers aged 18 to 25

In [34]:
top_3_categories = purchases_share_df.orderBy(col("spending_share_18_25").desc()).limit(3)
top_3_categories.show()

+-----------+---------------------+---------------------+--------------------+
|   category|total_purchases_18_25|total_purchase_amount|spending_share_18_25|
+-----------+---------------------+---------------------+--------------------+
|   Clothing|                245.0|                790.3|                31.0|
|     Sports|                310.5|               1802.5|               17.23|
|Electronics|                176.0|               1174.8|               14.98|
+-----------+---------------------+---------------------+--------------------+

